In [2]:
import pandas as pd
from multielo import MultiElo
from collections import namedtuple

STARTING_ELO = 1200
ELO_DICT = {}
RACES = []
DRIVERS = []

class Circuit:
    def __init__(self, circuitId, reference, name, location, country, wiki):
        self.id = circuitId
        self.reference = reference
        self.name = name
        self.location = location
        self.country = country
        self.wiki = wiki

    def __str__(self):
        return f'<Circuit {self.id}>'
    
    
class Race:
    def __init__(self, raceId, year, circuitId, name, date, wiki):
        self.id = raceId
        self.year = year
        self.circuitId = circuitId
        self.name = name
        self.date = date
        self.wiki = wiki
        self.drivers = []
        self.results = []
        #self.excluded = []
        
    def __str__(self):
        return f'<Race {self.id}>'

    
class Driver:
    def __init__(self, driverId, reference, number, code, firstName, lastName, dob, nationality, wiki):
        self.id = driverId
        self.reference = reference
        self.number = number
        self.code = code
        self.firstName = firstName
        self.lastName = lastName
        self.dob = dob
        self.nationality = nationality
        self.wiki = wiki
        self.elo = STARTING_ELO
        self.ratingChanges = {}
    
    def __str__(self):
        return f'<Driver {self.id}>'
    
    def ChangeRating(self, race, elo):
        self.ratingChanges[race] = (self.elo, elo)
        self.elo = elo
    
    
def DriverRankingDelta(driver, race):
    if (driver.id not in race.drivers):
        raise ValueError("Driver did not participate in race.")
    else:
        return driver.ratingChanges[race.id][1] - driver.ratingChanges[race.id][0]

In [3]:
# Driver data loading
df_drivers = pd.read_csv('data/drivers.csv')
drivers = []
for index, row in df_drivers.iterrows():
    driver = Driver(row['driverId'], row['driverRef'], row['number'], row['code'], row['forename'], row['surname'], row['dob'], row['nationality'], row['url'])
    drivers.append(driver)           

for driver in drivers:
    ELO_DICT[driver.id] = 1200
DRIVERS = drivers

In [4]:
# Race data loading
df_races = pd.read_csv('data/races.csv')
races = []
for index, row in df_races.iterrows():
    race = Race(row['raceId'], row['year'], row['circuitId'], row['name'], row['date'], row['url'])
    races.append(race)

RACES = races

In [5]:
# Circuit data loading
df_circuits = pd.read_csv('data/circuits.csv')
circuits = []
for index, row in df_circuits.iterrows():
    circuit = Circuit(row['circuitId'], row['circuitRef'], row['name'], row['location'], row['country'], row['url'])
    circuits.append(circuit)
print(circuits[0])

<Circuit 1>


In [6]:
# Result data loading
df_results = pd.read_csv('data/results.csv')
df_results.head()

,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId
0,1,18,1,1,22,1,1,1,1,10.0,58,1:34:50.616,5690616,39,2,1:27.452,218.300,1
1,2,18,2,2,3,5,2,2,2,8.0,58,+5.478,5696094,41,3,1:27.739,217.586,1
2,3,18,3,3,7,7,3,3,3,6.0,58,+8.163,5698779,41,5,1:28.090,216.719,1
3,4,18,4,4,5,11,4,4,4,5.0,58,+17.181,5707797,58,7,1:28.603,215.464,1
4,5,18,5,1,23,3,5,5,5,4.0,58,+18.014,5708630,43,1,1:27.418,218.385,1


In [7]:
for race in RACES:
    id = race.id
    df = df_results.loc[df_results['raceId'] == id].sort_values(by='positionOrder')
    drivers = []
    for index, row in df.iterrows():
        drivers.append((int(row['driverId']),int(row['positionOrder'])))

    drivers.sort(key=lambda x: x[1])

    for d,r in drivers:
        race.drivers.append(d)
        race.results.append(r)

In [8]:
driver_dict = {}
race_dict = {}
for driver in DRIVERS:
    driver_dict[driver.id] = driver
for race in RACES:
    race_dict[race.id] = race

In [9]:
# reset ELO_DICT
#for driver in DRIVERS:
#    ELO_DICT[driver.id] = 1200
#ELO_DICT

In [10]:
elo = MultiElo()
def getResultArray(race):
    arr = []
    for driver in race.drivers:
        arr.append(ELO_DICT[driver])
    return arr
    
for race in races:
    try:
        result = getResultArray(race)
        new = elo.get_new_ratings(result)
        for i,x in enumerate(new):
            ELO_DICT[race.drivers[i]] = x
            driver_dict[race.drivers[i]].ChangeRating(race,x)
    except:
        print(f"Failed Race ID: {race.id}")

Failed Race ID: 1074
Failed Race ID: 1075
Failed Race ID: 1076
Failed Race ID: 1077
Failed Race ID: 1078
Failed Race ID: 1079
Failed Race ID: 1080
Failed Race ID: 1081
Failed Race ID: 1082
Failed Race ID: 1083
Failed Race ID: 1084
Failed Race ID: 1085
Failed Race ID: 1086
Failed Race ID: 1087
Failed Race ID: 1088
Failed Race ID: 1089
Failed Race ID: 1090
Failed Race ID: 1091
Failed Race ID: 1092
Failed Race ID: 1093
Failed Race ID: 1094
Failed Race ID: 1095
Failed Race ID: 1096


In [11]:
ranking = []
for key in ELO_DICT:
    ranking.append((key, ELO_DICT[key]))

ranking.sort(key=lambda x: x[1], reverse=True)
for i,rank in enumerate(ranking):
    pos = i+1
    fname = driver_dict[rank[0]].firstName
    lname = driver_dict[rank[0]].lastName
    elo = rank[1]
    print(f'{pos}. {fname} {lname} - {elo}')

1. Lewis Hamilton - 1736.3119435568615
2. Nico Rosberg - 1669.961404686599
3. Max Verstappen - 1597.6238368811835
4. Valtteri Bottas - 1506.2219426612285
5. Jacques Villeneuve - 1504.810424774383
6. Carlos Sainz - 1484.013124131847
7. Mark Webber - 1434.9747534743585
8. Charles Leclerc - 1432.729986778366
9. Bruce McLaren - 1432.598435513201
10. Phil Hill - 1432.4661526677646
11. Lando Norris - 1431.1119610229591
12. Juan Fangio - 1426.7152536450092
13. Gilles Villeneuve - 1421.2573165295746
14. Sergio Pérez - 1412.4182525057997
15. Nino Farina - 1411.7661661367313
16. Richie Ginther - 1408.8511500503294
17. Alexander Albon - 1405.7735131711565
18. Clay Regazzoni - 1404.8371888336883
19. Patrick Depailler - 1400.4080818961174
20. Jim Clark - 1394.7442639158792
21. Daniel Ricciardo - 1393.0306764159147
22. Mike Hawthorn - 1389.8749437763984
23. David Coulthard - 1388.7270999442824
24. Didier Pironi - 1382.5308645812481
25. Maurício Gugelmin - 1381.6243420585672
26. Jean Alesi - 1378.373

In [12]:
# Track elo changes
driver_dict[30].ratingChanges

{<__main__.Race at 0x7ff6a3b16e48>: (1200, 1228.9037995769368),
 <__main__.Race at 0x7ff6a3b16e80>: (1228.9037995769368, 1243.7277076537052),
 <__main__.Race at 0x7ff6a3b16eb8>: (1243.7277076537052, 1225.3471264930176),
 <__main__.Race at 0x7ff6a3b16ef0>: (1225.3471264930176, 1255.0574498647593),
 <__main__.Race at 0x7ff6a3b16f28>: (1255.0574498647593, 1282.4826517024937),
 <__main__.Race at 0x7ff6a3b16f60>: (1282.4826517024937, 1304.7279135671506),
 <__main__.Race at 0x7ff6a3b16f98>: (1304.7279135671506, 1316.4796111172316),
 <__main__.Race at 0x7ff6a3b16fd0>: (1316.4796111172316, 1336.0499919216234),
 <__main__.Race at 0x7ff6a3fa0048>: (1336.0499919216234, 1354.1788874686374),
 <__main__.Race at 0x7ff6a3fa0080>: (1354.1788874686374, 1373.8941334164299),
 <__main__.Race at 0x7ff6a3fa00b8>: (1373.8941334164299, 1392.1777216593791),
 <__main__.Race at 0x7ff6a3fa00f0>: (1392.1777216593791, 1409.2916344265018),
 <__main__.Race at 0x7ff6a3fa0128>: (1409.2916344265018, 1405.456443360823),
 

In [13]:
# 2014+ rankings
ranking = []
for key in ELO_DICT:
    ranking.append((key, ELO_DICT[key]))

ranking.sort(key=lambda x: x[1], reverse=True)
counter = 1
for i,rank in enumerate(ranking):
    if driver_dict[rank[0]].number != '\\N':
        pos = counter
        fname = driver_dict[rank[0]].firstName
        lname = driver_dict[rank[0]].lastName
        elo = rank[1]
        print(f'{pos}. {fname} {lname} - {elo}')
        counter+=1

1. Lewis Hamilton - 1736.3119435568615
2. Nico Rosberg - 1669.961404686599
3. Max Verstappen - 1597.6238368811835
4. Valtteri Bottas - 1506.2219426612285
5. Carlos Sainz - 1484.013124131847
6. Charles Leclerc - 1432.729986778366
7. Lando Norris - 1431.1119610229591
8. Sergio Pérez - 1412.4182525057997
9. Alexander Albon - 1405.7735131711565
10. Daniel Ricciardo - 1393.0306764159147
11. Pierre Gasly - 1377.6181706732038
12. Felipe Massa - 1325.7454902748925
13. Esteban Ocon - 1323.1257809166743
14. Fernando Alonso - 1313.7761756271047
15. Daniil Kvyat - 1298.0876010099817
16. Sebastian Vettel - 1267.4929641935096
17. Nico Hülkenberg - 1263.5736080835256
18. Lance Stroll - 1242.9782266070918
19. Jean-Éric Vergne - 1225.3817407492058
20. Kimi Räikkönen - 1209.9594498630452
21. Yuki Tsunoda - 1205.568951221904
22. Antonio Giovinazzi - 1201.6068199272477
23. Stoffel Vandoorne - 1199.795584727886
24. Jenson Button - 1196.7181521961384
25. Jack Aitken - 1190.8058966871683
26. Pastor Maldonado